# Importing libraries

In [1]:
import os
import imp
import time
import torch
import nilearn
import numpy as np
from tqdm import tqdm
from nilearn import image
from scipy.ndimage.morphology import binary_dilation, binary_erosion

import utils.data_processor as data_processor
imp.reload(data_processor)
from utils.data_processor import *

import utils.visualization_tools as visualization_tools
imp.reload(visualization_tools)
from utils.visualization_tools import *

%config Completer.use_jedi = False

# Creating lists with paths

In [2]:
def get_paths(allowed_subjects=None):
    
    if not allowed_subjects:
        allowed_subdirs = [f"sub-{name[:-4]}" for name in os.listdir(f"raw_data/normalized_label")]
    else:
        allowed_subdirs = [f'sub-{name}' for name in allowed_subjects]
    
    brains = [[f"raw_data/output/fmriprep/{subdir}/anat/{name}" for name in 
                   os.listdir(f"raw_data/output/fmriprep/{subdir}/anat") if 
                       name.endswith("Asym_desc-preproc_T1w.nii.gz")] for subdir in
                            os.listdir(f"raw_data/output/fmriprep/") if subdir.count(".") == 0 and
                                  subdir.startswith("sub") and subdir in allowed_subdirs]
    brains = [item for sublist in brains for item in sublist]

    brain_masks = [[f"raw_data/output/fmriprep/{subdir}/anat/{name}" for name in
                        os.listdir(f"raw_data/output/fmriprep/{subdir}/anat") if
                            name.endswith("Asym_desc-brain_mask.nii.gz")] for subdir in
                               os.listdir(f"raw_data/output/fmriprep/") if subdir.count(".") == 0 and
                                   subdir.startswith("sub") and subdir in allowed_subdirs]
    brain_masks = [item for sublist in brain_masks for item in sublist]
    
    brain_nums = [x.split('/')[3][4:] for x in brains]
        
    labels = [f"raw_data/normalized_label/{name}.nii" for name in brain_nums]
    brain_masks_aseg = [f"raw_data/normalized_segmentation/{name}.nii" for name in brain_nums]
    
    # additional features
    curv = [f"raw_data/volums_features/curv/norm-{name}.nii" for name in brain_nums]
    thickness = [f"raw_data/volums_features/thickness/norm-{name}.nii" for name in brain_nums]
    sulc = [f"raw_data/volums_features/sulc/norm-{name}.nii" for name in brain_nums]
    
    return brains, brain_masks, brain_nums, labels, brain_masks_aseg, curv, thickness, sulc

In [3]:
allowed_subjects = np.load('raw_data/sub_with_all_data.npy', allow_pickle=True).tolist()
paths = get_paths(allowed_subjects)
brains, brain_masks, brain_nums, labels, brain_masks_aseg, curvs, thickness, sulcs = paths

# Resampling to the same size

In [4]:
def find_number_of_erosion(mask):
    i = 0
    tmp = mask.copy()
    while True:
        if tmp.any() == False:
            return i
        tmp = binary_erosion(tmp)
        i += 1

In [5]:
number_of_brains = len(brains)
nib_brains = [nib.load(brains[i]) for i in range(number_of_brains)]

good_shape = (241, 336, 283)
indexes_of_brain_with_good_shape = [i for i in range(len(nib_brains)) if nib_brains[i].shape==good_shape]
good_brain = nib_brains[indexes_of_brain_with_good_shape[0]]

for brain_num, brain, brain_mask, label,\
    brain_mask_aseg, curv, thick, sulc in tqdm(zip(brain_nums, brains, brain_masks,
                                                     labels, brain_masks_aseg, curvs,
                                                     thickness, sulcs), total = number_of_brains):
    nib_brain = nib.load(brain)
    nib_brain_mask = nib.load(brain_mask)
    nib_label = nib.load(label)
    nib_brain_mask_aseg = nib.load(brain_mask_aseg)
    nib_curv = nib.load(curv)
    nib_thick = nib.load(thick)
    nib_sulc = nib.load(sulc)
    
    nib_brain = nilearn.image.resample_to_img(nib_brain, good_brain)
    nib_brain_mask = nilearn.image.resample_to_img(nib_brain_mask, good_brain, interpolation = 'nearest')
    arr_skull_striped = nib_brain.get_fdata().astype(np.int16) * nib_brain_mask.get_fdata().astype(np.int8)
    nib_skull_striped = nib.Nifti1Image(arr_skull_striped, nib_brain.affine)
    nib_label = nilearn.image.resample_to_img(nib_label, good_brain)
    nib_brain_mask_aseg = nilearn.image.resample_to_img(nib_brain_mask_aseg, good_brain, interpolation = 'nearest')
    nib_curv = nilearn.image.resample_to_img(nib_curv, good_brain)
    nib_thick = nilearn.image.resample_to_img(nib_thick, good_brain)
    nib_sulc = nilearn.image.resample_to_img(nib_sulc, good_brain)
    
    nib.save(nib_label, f'dataset_ready_to_use/data_full/labels/{brain_num}.nii')
    nib.save(nib_skull_striped, f'dataset_ready_to_use/data_full/brains/{brain_num}.nii')
    nib.save(nib_brain_mask_aseg, f'dataset_ready_to_use/data_full/segmentations/{brain_num}.nii')
    nib.save(nib_curv, f'dataset_ready_to_use/data_full/curv/{brain_num}.nii')
    nib.save(nib_thick, f'dataset_ready_to_use/data_full/thickness/{brain_num}.nii')
    nib.save(nib_sulc, f'dataset_ready_to_use/data_full/sulc/{brain_num}.nii')

    arr_brain = nib_brain.get_fdata()
    arr_brain_mask = nib_brain_mask.get_fdata().astype(np.int8)
    arr_label = nib_label.get_fdata().astype(np.int8)
    arr_brain_mask_aseg = nib_brain_mask_aseg.get_fdata()
    arr_curv = nib_curv.get_fdata()
    arr_thick = nib_thick.get_fdata()
    arr_sulc = nib_sulc.get_fdata()
    
    del nib_brain, nib_brain_mask, nib_label, nib_brain_mask_aseg, nib_curv, nib_thick, nib_sulc
    
    arr_label = (arr_label>0).astype(np.int8)
    arr_brain = arr_brain * arr_brain_mask
    
    arr_brain_mask_aseg_hypo = np.logical_or((arr_brain_mask_aseg == 17), (arr_brain_mask_aseg == 53)).astype(np.int8)
    arr_brain_mask_aseg_putamen = np.logical_or((arr_brain_mask_aseg == 12), (arr_brain_mask_aseg == 51)).astype(np.int8)
    arr_brain_mask_aseg_grey = (arr_brain_mask_aseg >= 1000).astype(np.int8)
    
    grey_width = find_number_of_erosion(arr_brain_mask_aseg_grey)
    hypo_width = find_number_of_erosion(arr_brain_mask_aseg_hypo) 
    
    grey_dilation = int(0.6 * grey_width) 
    hypo_dilation = int(round(0.1 * hypo_width, 0))
    
    grey_mask_dilated = binary_dilation(arr_brain_mask_aseg_grey, iterations = grey_dilation)
    hypo_mask_dilated = binary_dilation(arr_brain_mask_aseg_hypo, iterations = hypo_dilation)
    
    final_mask = np.logical_or(grey_mask_dilated, hypo_mask_dilated)
    final_mask = np.logical_or(final_mask, arr_brain_mask_aseg_putamen)
    final_mask = np.logical_and(final_mask, arr_brain_mask) 
    
    del arr_brain_mask_aseg_hypo, arr_brain_mask_aseg_putamen,\
    arr_brain_mask_aseg_grey, grey_mask_dilated, hypo_mask_dilated, arr_brain_mask, arr_brain_mask_aseg
    
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/full/brains/{brain_num}.npy', arr_brain*final_mask)
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/full/labels/{brain_num}.npy', arr_label*final_mask)
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/full/curv/{brain_num}.npy', arr_curv*final_mask)
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/full/thickness/{brain_num}.npy', arr_thick*final_mask)
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/full/sulc/{brain_num}.npy', arr_sulc*final_mask)
    
    temple_zone_mask = np.zeros(arr_brain.shape, dtype = np.int8)
    temple_zone_mask[:, :, :65] = 1.
    
    final_mask_temple = np.logical_and(final_mask, temple_zone_mask)
    final_mask_nottemple = np.logical_and(final_mask, 1 - temple_zone_mask)
    
    del temple_zone_mask
    
    brain_aseg_temple = arr_brain * final_mask_temple
    brain_aseg_nottemple = arr_brain * final_mask_nottemple
    
    label_aseg_temple = arr_label * final_mask_temple
    label_aseg_nottemple = arr_label * final_mask_nottemple
    
    curv_aseg_temple = arr_curv * final_mask_temple
    curv_aseg_nottemple = arr_curv * final_mask_nottemple
    
    thick_aseg_temple = arr_thick * final_mask_temple
    thick_aseg_nottemple = arr_thick * final_mask_nottemple
    
    sulc_aseg_temple = arr_sulc * final_mask_temple
    sulc_aseg_nottemple = arr_sulc * final_mask_nottemple
    
    del arr_brain, arr_label, arr_curv, arr_thick, arr_sulc, final_mask_temple, final_mask_nottemple 
    
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/temple/brains/{brain_num}.npy', brain_aseg_temple)
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/nottemple/brains/{brain_num}.npy', brain_aseg_nottemple)
    
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/temple/labels/{brain_num}.npy', label_aseg_temple)
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/nottemple/labels/{brain_num}.npy', label_aseg_nottemple)
    
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/temple/curv/{brain_num}.npy', curv_aseg_temple)
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/nottemple/curv/{brain_num}.npy', curv_aseg_nottemple)
    
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/temple/thickness/{brain_num}.npy', thick_aseg_temple)
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/nottemple/thickness/{brain_num}.npy', thick_aseg_nottemple)
    
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/temple/sulc/{brain_num}.npy', sulc_aseg_temple)
    np.save(f'dataset_ready_to_use/data_only_usefull_areas/nottemple/sulc/{brain_num}.npy', sulc_aseg_nottemple)
    
    del brain_aseg_temple, brain_aseg_nottemple,\
        label_aseg_temple, label_aseg_nottemple,\
        curv_aseg_temple, curv_aseg_nottemple,\
        thick_aseg_temple, thick_aseg_nottemple,\
        sulc_aseg_temple, sulc_aseg_nottemple

100%|██████████| 81/81 [3:46:53<00:00, 168.07s/it]  
